In [1]:
import gensim
from gensim.models.callbacks import LossLogger, LossSetter
from gensim.models.keyedvectors import PoincareWordEmbeddingsKeyedVectors as pkv
from gensim.matutils import argsort
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from numpy import dot
from operator import itemgetter
import os
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import RidgeCV
from scipy import stats
random.seed(42)

ROOT = "/Users/alext/Documents/Master/Thesis/"

In [2]:
# hyp_model_fn = os.path.join(ROOT, "models/geometric_emb/w2v_levy_nll_5_100_A01_a01_n5_w5_c100_poincare_OPTwfullrsgd_burnin1_cosh")
hyp_model_fn = os.path.join(ROOT, "models/geometric_emb/w2v_levy_nll_5_100_A01_a0001_n5_w5_c100_poincare_OPTwfullrsgd_SIMcosh-dist-sq_burnin1")
hyp_model = gensim.models.Word2Vec.load(hyp_model_fn)
hyp_wv = hyp_model.wv

In [3]:
w2v_model_fn = os.path.join(ROOT, "models/word2vec_baseline/w2v_levy_sg_5_100_A025_a0001_n5_w5_c100_cosine_OPTsgd")
w2v_model = gensim.models.Word2Vec.load(w2v_model_fn)
w2v_wv = w2v_model.wv

In [19]:
split_type = "lexical"
hyperlex_training_file = os.path.join(ROOT, "data/hyperlex-data/splits/"+split_type+"/hyperlex_training_all_"+split_type+".txt")
hyperlex_validation_file = os.path.join(ROOT, "data/hyperlex-data/splits/"+split_type+"/hyperlex_dev_all_"+split_type+".txt")
hyperlex_test_file = os.path.join(ROOT, "data/hyperlex-data/splits/"+split_type+"/hyperlex_test_all_"+split_type+".txt")

def read_hyperlex_format(filename):
    with open(filename, "r") as f:
        lines = [line.strip().split() for line in f.readlines()[1:]]
        result = []
        discarded_count = 0
        for line in lines:
            if line[0] not in hyp_wv.vocab or line[1] not in hyp_wv.vocab:
                discarded_count += 1
                continue
            result.append([line[0], line[1], hyp_wv.vocab[line[0]].index, hyp_wv.vocab[line[1]].index, float(line[5])])
        print("Discarded {} pairs out of {}".format(discarded_count, len(lines)))
        return np.array(result)

train_set = read_hyperlex_format(hyperlex_training_file)
valid_set = read_hyperlex_format(hyperlex_validation_file)
test_set = read_hyperlex_format(hyperlex_test_file)

print(train_set[:10])

Discarded 2 pairs out of 1133
Discarded 0 pairs out of 85
Discarded 1 pairs out of 269
[['trail' 'follow' '2318' '1724' '7.18']
 ['mason' 'worker' '5595' '5323' '7.5']
 ['radish' 'carrot' '66093' '25733' '0.15']
 ['veal' 'meat' '48465' '4046' '9.77']
 ['keyboard' 'arm' '5948' '3050' '0.9']
 ['swing' 'push' '6408' '4734' '3.05']
 ['rhyme' 'dinner' '15487' '5465' '0.0']
 ['father' 'ancestor' '386' '8117' '6.5']
 ['truce' 'peace' '13161' '1351' '8.0']
 ['glass' 'cabinet' '2244' '2820' '3.2']]


## Ridge regression on vectors

In [20]:
def gyr_midpoint(u, v):    
    gamma_u_sq = 1 / (1 - norm(u)**2)
    gamma_v_sq = 1 / (1 - norm(v)**2)
    denominator = gamma_u_sq + gamma_v_sq - 1
    agg = u * gamma_u_sq / denominator + v * gamma_v_sq / denominator

    return hyp_wv.moebius_mul(agg, 0.5)

def agg_emb(word):
    w = hyp_wv.word_vec(word)
    c = hyp_wv.trainables.syn1neg[hyp_wv.vocab[word].index]
    
    gamma_w_sq = 1 / (1 - norm(w)**2)
    gamma_c_sq = 1 / (1 - norm(c)**2)
    denominator = gamma_w_sq + gamma_c_sq - 1
    agg = w * gamma_w_sq / denominator + c * gamma_c_sq / denominator

    return hyp_wv.moebius_mul(agg, 0.5)


def extract_vector_features(dataset):
    word0 = dataset[:, 0]
    word1 = dataset[:, 1]
    index0 = dataset[:, 2].astype(int)
    index1 = dataset[:, 3].astype(int)
    labels = dataset[:, 4].astype(float)
    
    # Use w (target vectors).
    hyp_features = np.array([v1 - v2 for v1, v2 in zip(hyp_model.wv.vectors[index0], hyp_model.wv.vectors[index1])])
    w2v_features = np.array([v1 - v2 for v1, v2 in zip(w2v_model.wv.vectors[index0], w2v_model.wv.vectors[index1])])

    # Use c (context vectors).
#     hyp_features = np.array([v1 - v2 for v1, v2 in zip(hyp_model.trainables.syn1neg[index0], 
#                                                        hyp_model.trainables.syn1neg[index1])])
#     w2v_features = np.array([v1 - v2 for v1, v2 in zip(w2v_model.trainables.syn1neg[index0], 
#                                                        w2v_model.trainables.syn1neg[index1])])
    
    # Use w+c (combination of target and context vectors).
#     hyp_features = np.array([agg_emb(w0) - agg_emb(w1) for w0, w1 in zip(word0, word1)])
#     w2v_features = np.array([((w2v_model.wv.vectors[i0] + w2v_model.trainables.syn1neg[i0]) - 
#                               (w2v_model.wv.vectors[i1] + w2v_model.trainables.syn1neg[i1])) 
#                              for i0, i1 in zip(index0, index1)])

    print(hyp_features.shape, w2v_features.shape)
    return w2v_features, hyp_features, labels


def eval_model(model, emb_type, test=False):
    if emb_type == "w2v":
        train_features = w2v_train_features
        valid_features = w2v_valid_features
        test_features = w2v_test_features
    elif emb_type == "hyp":
        train_features = hyp_train_features
        valid_features = hyp_valid_features
        test_features = hyp_test_features
    if test == False:
        train_pred = model.predict(train_features[:, :])
        diff = abs(train_pred - train_labels)
        print("TRAIN", diff[:10], min(diff), max(diff))
        print("TRAIN", np.average(diff))
        print("TRAIN", stats.spearmanr(train_pred, train_labels))

        print()
        valid_pred = model.predict(valid_features[:, :])
        diff = abs(valid_pred - valid_labels)
        print("VALID", diff[:10], min(diff), max(diff))
        print("VALID", np.average(diff))
        print("VALID", stats.spearmanr(valid_pred, valid_labels))
    else:
        test_pred = model.predict(test_features[:, :])
        diff = abs(test_pred - test_labels)
        print("TEST", diff[:10], min(diff), max(diff))
        print("TEST", np.average(diff))
        print("TEST", stats.spearmanr(test_pred, test_labels))

w2v_train_features, hyp_train_features, train_labels = extract_vector_features(train_set)
w2v_valid_features, hyp_valid_features, valid_labels = extract_vector_features(valid_set)
w2v_test_features, hyp_test_features, test_labels = extract_vector_features(test_set)

(1131, 100) (1131, 100)
(85, 100) (85, 100)
(268, 100) (268, 100)


In [21]:
def run_ridge_model(emb_type):
    if emb_type == "w2v":
        train_features = w2v_train_features
        valid_features = w2v_valid_features
        test_features = w2v_test_features
    elif emb_type == "hyp":
        train_features = hyp_train_features
        valid_features = hyp_valid_features
        test_features = hyp_test_features

    model = RidgeCV(cv=3, alphas=[0.5, 0.1, 0.05, 0.01, 0.001], fit_intercept=True)
    model.fit(train_features, train_labels)
    return model

In [22]:
w2v_ridge = run_ridge_model("w2v")

In [23]:
eval_model(w2v_ridge, emb_type="w2v")

TRAIN [0.83931225 0.02306938 5.89142189 2.27025225 3.12085972 2.20447607
 5.5392766  0.27051163 1.58162498 0.79486923] 0.006312990188598544 7.472495079040527
TRAIN 2.3567746885301366
TRAIN SpearmanrResult(correlation=0.5580588339411088, pvalue=1.4118650649762757e-93)

VALID [2.22260941 4.02918327 4.43202002 1.88520439 3.34023502 3.36789791
 1.82594969 0.10131618 3.52245209 0.63070992] 0.02180459976196314 7.190348148345947
VALID 2.816758682363173
VALID SpearmanrResult(correlation=0.2518542411567876, pvalue=0.020060337630393946)


In [24]:
hyp_ridge = run_ridge_model("hyp")

In [25]:
eval_model(hyp_ridge, emb_type="hyp")

TRAIN [0.46425184 0.82849733 4.08148089 2.36706749 3.94460292 2.42400949
 3.95069928 0.74840216 2.68918105 2.04660479] 0.009061480353242946 7.306734142485243
TRAIN 2.286842027485845
TRAIN SpearmanrResult(correlation=0.5947496099118361, pvalue=3.8042197876010737e-109)

VALID [0.81900807 3.15307244 3.03276536 3.44950212 3.8165348  3.70392245
 0.53857867 0.39779811 2.23337805 0.68223118] 0.16826914916853752 8.47892088736251
VALID 2.5648639340921044
VALID SpearmanrResult(correlation=0.438226184172434, pvalue=2.7331100261430763e-05)


In [26]:
eval_model(w2v_ridge, emb_type="w2v", test=True)
eval_model(hyp_ridge, emb_type="hyp", test=True)
print(test_labels[:10])

TEST [0.47307289 0.02393093 6.20002186 1.88935013 0.68855192 3.97857544
 0.0470562  2.07115429 2.57394838 0.27788012] 0.008690147399902592 9.206815643310547
TEST 2.7133184649339364
TEST SpearmanrResult(correlation=0.2568577781719385, pvalue=2.0725756647114863e-05)
TEST [1.46365169 0.65804693 4.93491284 2.87132056 0.91628803 3.67058776
 0.01718816 1.03374268 3.65657208 0.13324443] 0.012401410909948396 9.354079831298314
TEST 2.54585703796331
TEST SpearmanrResult(correlation=0.3325358954395362, pvalue=2.4337653773623352e-08)
[2.37 6.15 0.13 7.95 2.73 8.72 3.5  7.18 4.   4.08]


## Regression to get Hypernymy Score
**Features** (for a pair of words v, w): target/context norms, target/context cosh(distance) between them, target/context "inner product"

In [5]:
def extract_features(dataset):
    word0 = dataset[:, 0]
    word1 = dataset[:, 1]
    index0 = dataset[:, 2].astype(int)
    index1 = dataset[:, 3].astype(int)
    labels = dataset[:, 4].astype(float)
    
    all_features = []
    for model in [w2v_model, hyp_model]:
        features = []
        features.append(norm(model.wv.vectors[index0], axis=1))
        features.append(norm(model.wv.vectors[index1], axis=1))
        features.append(norm(model.trainables.syn1neg[index0], axis=1))
        features.append(norm(model.trainables.syn1neg[index1], axis=1))
        features.append([model.wv.distance(v0, v1) for v0, v1 in zip(model.wv.vectors[index0], model.wv.vectors[index1])])
        features.append([model.wv.distance(v0, v1) 
                         for v0, v1 in zip(model.trainables.syn1neg[index0], 
                                           model.trainables.syn1neg[index1])])
        features = np.array(features).T
        print(features.shape)
        all_features.append(features)
    # TODO: maybe add "inner product"
    return all_features[0], all_features[1], labels
    
w2v_train_features, hyp_train_features, train_labels = extract_features(train_set)
w2v_valid_features, hyp_valid_features, valid_labels = extract_features(valid_set)
w2v_test_features, hyp_test_features, test_labels = extract_features(test_set)
print(hyp_train_features[:10])

(1828, 6)
(1828, 6)
(130, 6)
(130, 6)
(654, 6)
(654, 6)
[[0.51168466 0.47334309 0.55542843 0.53295237 1.19702859 1.56273169]
 [0.50451891 0.4853155  0.54614616 0.5097985  1.27352404 1.42389357]
 [0.47089291 0.48295117 0.49271411 0.52365555 1.45510096 1.56716323]
 [0.50506682 0.4845155  0.54587093 0.49841875 1.5309626  1.77439715]
 [0.49187483 0.46320014 0.54483784 0.50267289 1.18837734 1.44291092]
 [0.51368189 0.47550974 0.58919611 0.53441454 0.81370259 1.08290528]
 [0.50733578 0.49104007 0.53141288 0.53973863 0.7831726  0.98292347]
 [0.48849577 0.47570591 0.55038228 0.51972331 1.50600329 1.87014381]
 [0.44302898 0.48320492 0.47343314 0.52305128 1.29184253 1.47893638]
 [0.47828938 0.44166958 0.51515283 0.46249405 1.09883588 1.11704733]]


In [6]:
def run_model(emb_type):
    if emb_type == "w2v":
        train_features = w2v_train_features
        valid_features = w2v_valid_features
        test_features = w2v_test_features
    elif emb_type == "hyp":
        train_features = hyp_train_features
        valid_features = hyp_valid_features
        test_features = hyp_test_features

#     model = MLPRegressor(
#         max_iter=5000, shuffle=True,tol=1e-5, verbose=False, warm_start=False,
#         early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#     hidden_layer_sizes = [(int(x),int(y),) for x, y in zip(np.linspace(start = 10, stop = 200, num = 10),
#                                                            np.linspace(start = 10, stop = 200, num = 20))]
# #     hidden_layer_sizes = [(int(x),) for x in np.linspace(start = 10, stop = 200, num = 10)]
#     random_grid = {
#         'hidden_layer_sizes': hidden_layer_sizes,
#         'activation': ['relu', 'tanh', 'logistic'],
# #         'solver': ['adam', 'lbfgs', 'sgd'],
# #         'alpha': [0.1, 0.01, 0.001, 0.0001],
# #         'learning_rate': ['constant', 'invscaling', 'adaptive'],
# #         'learning_rate_init': [0.1, 0.01, 0.001, 0.0001],
# #         'power_t': [0.5, 0.1, 0.05],
# #         'momentum': [0.9, 0.5, 0.1],
# #         'nesterovs_momentum': [True, False],
#     }
    
    model = RandomForestRegressor()
    random_grid = {
        'n_estimators': [int(x) for x in np.linspace(start = 10, stop = 200, num = 10)],
        'max_features': ['auto', 'sqrt'],
        'max_depth': [int(x) for x in np.linspace(3, 100, num = 10)] + [None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False],
    }

    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, n_iter = 200, cv = 3, 
                                   verbose=0, random_state=42, n_jobs=8)
    # Fit the random search model
    rf_random.fit(train_features, train_labels)
    return rf_random

In [7]:
w2v_regressor = run_model("w2v")

In [8]:
print(w2v_regressor.best_params_)
eval_model(w2v_regressor.best_estimator_, emb_type="w2v")

{'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 46, 'bootstrap': True}
TRAIN [2.81668435 1.49772388 2.5022517  2.72115281 1.0969728  2.48602387
 2.53055243 2.3754695  0.81412209 1.8807676 ] 0.001029985036033132 4.7199860974560375
TRAIN 1.6267927788140857
TRAIN SpearmanrResult(correlation=0.9054006264159385, pvalue=0.0)

VALID [1.54939151 4.86339848 2.28249996 0.58930781 4.93321819 0.63282627
 0.21139278 1.9299245  4.16205995 3.30418047] 0.007708295108050045 7.4999793891974
VALID 2.736487022273141
VALID SpearmanrResult(correlation=0.29772251921151044, pvalue=0.0005814417613288742)


In [9]:
hyp_regressor = run_model("hyp")

In [10]:
print(hyp_regressor.best_params_)
eval_model(hyp_regressor.best_estimator_, emb_type="hyp")

{'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 56, 'bootstrap': True}
TRAIN [1.88475 1.6824  1.04765 0.5048  0.07775 0.79745 1.57225 0.0061  0.89535
 1.53875] 0.0004499999999953985 3.6570499999999937
TRAIN 0.9303531728665214
TRAIN SpearmanrResult(correlation=0.9744807356959447, pvalue=0.0)

VALID [0.5731  4.8533  4.37845 0.58195 2.8488  2.21225 1.2625  2.29435 2.36935
 5.80285] 0.02499999999999636 5.8028499999999985
VALID 2.5209765384615386
VALID SpearmanrResult(correlation=0.37846962569906684, pvalue=9.002428538604692e-06)


In [11]:
eval_model(w2v_regressor.best_estimator_, emb_type="w2v", test=True)
eval_model(hyp_regressor.best_estimator_, emb_type="hyp", test=True)
print(test_labels[:10])

TEST [6.17893772 2.86252605 5.09430841 0.31743548 1.93127197 1.92234529
 0.40391572 3.81111291 0.58520266 3.0201177 ] 0.0017794688150698512 8.875442745196194
TEST 2.590600518632328
TEST SpearmanrResult(correlation=0.3946581417661118, pvalue=8.387308276123064e-26)
TEST [6.2168  3.03865 1.60135 0.2708  1.9309  1.28885 1.1853  0.9061  0.64555
 3.2867 ] 0.0007999999999999119 8.734949999999992
TEST 2.5345562691131502
TEST SpearmanrResult(correlation=0.3990734856473884, pvalue=2.137647789350276e-26)
[0.15 3.05 0.   3.2  8.97 0.52 5.77 2.82 6.18 8.83]
